In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data\preprocessed.csv")

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\USER\AppData\Local\Temp\ipykernel_13004\222109947.py:1: SyntaxWarning: invalid escape sequence '\p'
  df = pd.read_csv("data\preprocessed.csv")


In [3]:
df.columns

Index(['raw_id', 'clinic_application_date', 'intake_date', 'signing_date',
       'suiteable_for_pp', 'First name', 'sheet', 'clean_id', 'group',
       'therapy_start_date', 'therapy_end_date'],
      dtype='object')

In [4]:
priority = {
        'סיימו טיפול': 0,
        'CAU': 1,
        'IPC-SSC': 2,
        'משתתפים פעילים': 3,
        'פספוסי גיוסים': 4,
        'אין שת"פ טיפולי': 5,
        'אי התאמה למחקר': 6,
        'אי הסכמה למחקר': 7,
        'נשירה מחקרית': 8,
        'נשירה קלינית- לאחר ת. טיפול': 9,
        'עלייה לרמה 2': 10
}

In [5]:
formats = [
    "%Y-%m-%d",   # 2025-09-28
    "%d/%m/%Y",   # 28/09/2025
    "%m/%d/%Y",   # 09/28/2025
    "%d-%b-%Y",   # 28-Sep-2025
    "%d.%m.%Y",   # 28.09.2025
    '%Y-%m-%d %H:%M:%S',
    "%d.%m.%y",
    "YYYY-MM-DD HH:MM:SS",
    "%Y-%m-%d %H:%M:%S"



]

date_columns = [i for i in df.columns if 'date' in i]


In [6]:
def parse_date(x):
    for fmt in formats:
        try:
            if type(x) == str:
                x = x.strip()
            return pd.to_datetime(x, format=fmt)
        except (ValueError, TypeError):
            continue
    return pd.NaT

for date_col in date_columns:
    df[date_col] = df[date_col].apply(parse_date)

#### Filling missing data by prioraty 

In [7]:
# map priority and sort so preferred sources come first for each participant
df['prio'] = df['sheet'].map(priority)
df_sorted = df.sort_values(['clean_id', 'prio'])

# group by participant and take the first non-null value per column
# groupby.first() skips NA and returns first non-NA per column
result = df_sorted.groupby('clean_id', as_index=False).first()

# drop helper column if you like
result = result.drop(columns=['prio'])

In [8]:
dummies = pd.get_dummies(df['sheet']).astype(int)
dummies['clean_id'] = df['clean_id']
one_hot_g = dummies.groupby('clean_id', as_index=True).max().reset_index()

# merge similarly
final2 = result.merge(one_hot_g, on='clean_id', how='left')
#final2[dummies.columns] = final2[dummies.columns].fillna(0)

In [9]:
final2["first_contact_date"] = final2["intake_date"].fillna(final2["clinic_application_date"]).fillna(final2["signing_date"])
final2['therapy_starting_date'] =  final2["therapy_start_date"]#.fillna(final2["therapy_end_date"])

In [10]:
final2

,clean_id,raw_id,clinic_application_date,intake_date,signing_date,suiteable_for_pp,First name,sheet,group,therapy_start_date,...,אי התאמה למחקר,"אין שת""פ טיפולי",משתתפים פעילים,נשירה מחקרית,נשירה קלינית- לאחר ת. טיפול,סיימו טיפול,עלייה לרמה 2,פספוסי גיוסים,first_contact_date,therapy_starting_date
0,329822563,329822563,2025-03-16,2025-03-16,NaT,None,בר,נשירה מחקרית,CAU,2025-11-04,...,0,0,0,1,0,0,0,0,2025-03-16,2025-11-04
1,a0017,a0017s,2025-03-12,2025-04-06,NaT,1,עמית,פספוסי גיוסים,None,NaT,...,0,0,0,0,0,0,0,1,2025-04-06,NaT
2,a0226,a0226s,2025-01-07,2025-01-14,NaT,1,אדירם,אי התאמה למחקר,None,NaT,...,1,0,0,0,0,0,0,0,2025-01-14,NaT
3,a0531,a0531s,2025-01-21,2025-02-06,NaT,1,אבישי,אי הסכמה למחקר,CAU,NaT,...,0,0,0,0,0,0,0,0,2025-02-06,NaT
4,a0556,a0556s,2025-04-20,2025-05-11,NaT,1,עתי,פספוסי גיוסים,None,NaT,...,0,0,0,0,0,0,0,1,2025-05-11,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,y9000,y9000s,2024-09-23,2025-01-14,NaT,1,יובל,אי התאמה למחקר,None,2025-04-22,...,1,0,0,0,0,0,0,0,2025-01-14,2025-04-22
306,y9212,Y9212,2025-03-25,2025-03-25,2025-03-25,1,ירדן,משתתפים פעילים,CAU,NaT,...,0,0,1,0,0,0,0,0,2025-03-25,NaT
307,y9411,y9411s,2024-12-16,2025-03-20,NaT,None,ירון,נשירה קלינית- לאחר ת. טיפול,Stepped Care,2025-05-08,...,0,0,0,0,1,0,0,0,2025-03-20,2025-05-08
308,z1585,Z1585s,2025-02-27,2025-04-20,2025-04-20,לא- הומלץ רק שאלונים,זיו,משתתפים פעילים,Stepped Care,NaT,...,0,0,1,0,0,0,0,0,2025-04-20,NaT


In [11]:
final2.to_csv(r"data\processed_df_for_consort.csv", index=0)

In [12]:
final2['group'].unique()

array(['CAU', None, 'Stepped Care', 'טיפול בקבוצת הורים '], dtype=object)

In [13]:
final2.columns

Index(['clean_id', 'raw_id', 'clinic_application_date', 'intake_date',
       'signing_date', 'suiteable_for_pp', 'First name', 'sheet', 'group',
       'therapy_start_date', 'therapy_end_date', 'אי הסכמה למחקר',
       'אי התאמה למחקר', 'אין שת"פ טיפולי', 'משתתפים פעילים', 'נשירה מחקרית',
       'נשירה קלינית- לאחר ת. טיפול', 'סיימו טיפול', 'עלייה לרמה 2',
       'פספוסי גיוסים', 'first_contact_date', 'therapy_starting_date'],
      dtype='object')